In [ ]:
import numpy as np 
import pandas as pd 

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import os
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

from tqdm.notebook import tqdm_notebook

In [ ]:
train_images_path = '../input/petfinder-pawpularity-score/train/'
test_images_path = '../input/petfinder-pawpularity-score/test/'

train_pd = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_pd = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
train_pd.Id = [image_name + '.jpg' for image_name in train_pd.Id]
test_pd.Id = [image_name + '.jpg' for image_name in test_pd.Id]

# Data

In [ ]:
train_pd.head(10)

In [ ]:
train_pd.info()

In [ ]:
train_pd.describe()

In [ ]:
train_pd.corr()

In [ ]:
sns.heatmap(train_pd.corr(), 
        xticklabels=train_pd.columns[1:],
        yticklabels=train_pd.columns[1:])

In [ ]:
sns.histplot(train_pd.Pawpularity)

In [ ]:
most_pawpular = list(train_pd[train_pd.Pawpularity == 100].Id)
less_pawpular = list(train_pd[train_pd.Pawpularity < 10].Id)

In [ ]:
fig, axes = plt.subplots(2, 9, figsize=(20, 10))
for ax in axes.flat:
    ax.set_yticks([])
    ax.set_xticks([])
for i in range(18):
  axes[i//9, i%9].imshow(plt.imread(train_images_path + most_pawpular[i]))

In [ ]:
fig, axes = plt.subplots(2, 9, figsize=(20, 10))
for ax in axes.flat:
    ax.set_yticks([])
    ax.set_xticks([])
for i in range(18):
  axes[i//9, i%9].imshow(plt.imread(train_images_path + less_pawpular[i]))

# Dataset

In [ ]:
class ImagePandasDataset(Dataset):
    def __init__(self, pd_dataframe, img_name_column, img_dir, target_column=None, features_columns=None, transform=None):
        self.pd_dataframe = pd_dataframe
        self.img_name_column = img_name_column
        self.img_dir = img_dir
        self.target_column = target_column
        self.features_columns = features_columns
        self.transform = transform

    def __len__(self):
        return len(self.pd_dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.pd_dataframe[self.img_name_column].iloc[idx])
        image = Image.open(img_path)
        target = -1
        if self.target_column:
            target = self.pd_dataframe[self.target_column].iloc[idx]/100.
        if self.transform:
            image = self.transform(image)
        if self.features_columns:
            return image, torch.tensor(self.pd_dataframe[self.features_columns].iloc[idx]), target
        return image, target        

In [ ]:
FEATURES = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [ ]:
img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
])

train_ds = ImagePandasDataset(train_pd, 'Id', img_dir=train_images_path, 
                              target_column='Pawpularity', features_columns=FEATURES, transform=img_transforms)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)

In [ ]:
for image, features, label in train_loader:
    print(image.shape)
    print(features.shape)
    print(label.shape)
    break

In [ ]:
plt.imshow(image[0].permute(1, 2, 0).cpu())

# Model

In [ ]:
class CNNModel(nn.Module):
  def __init__(self):
    super().__init__()
    
    # cnn parts
    self.conv_head = nn.Sequential(
        nn.BatchNorm2d(3),
        nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=1),
 
        nn.BatchNorm2d(16),
        nn.Conv2d(16, 32, kernel_size=4, stride=1, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
 
        nn.Conv2d(32, 64, kernel_size=5, stride=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=3),
 
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=3),
    )
 
    self.image_decoder = nn.Sequential(
        nn.Flatten(),
        nn.BatchNorm1d(128*6*6),
        nn.Dropout(0.2),
        nn.Linear(in_features=128*6*6, out_features=512),
        nn.ReLU(),
 
        nn.Dropout(0.2),
        nn.Linear(in_features=512, out_features=256),
        nn.ReLU(),
        
        nn.Linear(in_features=256, out_features=128),
        nn.ReLU(),
 
        nn.Linear(in_features=128, out_features=32),
        nn.ReLU()
    )
     
    # mlp part
    self.linear_tail = nn.Sequential(
        nn.BatchNorm1d(32+12),
        nn.Dropout(0.2),
        nn.Linear(in_features=32+12, out_features=16),
        nn.ReLU(),
        
        nn.BatchNorm1d(16),
        nn.Linear(in_features=16, out_features=8),
        nn.ReLU(),
        
        nn.Linear(in_features=8, out_features=1),
        nn.ReLU()
    )
 
  def forward(self, image, features):
    img_coded = self.image_decoder(self.conv_head(image)) 
    return self.linear_tail(torch.cat((features, img_coded), -1))

# Training

In [ ]:
def random(x, mx=0.2):
    r = torch.normal(0.0, 1.0, size=x.size()).cuda()
    r = r / torch.max(r)
    return r * mx

def train(model, optimizer, scheduler, loss_fn):
  model.train()
  losses = []
  accs = []
  n = 0
  for values in tqdm_notebook(train_loader):
    x, features, target = values
    x, features, target = x.cuda(), features.cuda(), target.cuda()
    optimizer.zero_grad()
    target = target.unsqueeze(-1)

    pred = model(x, features)
    loss = loss_fn(pred.float(), target.float())
    loss.backward()
    
#     normal noise as augmentation
    noise_pred = model(x + random(x), features)
    noise_loss = loss_fn(noise_pred.float(), target.float())
    noise_loss.backward()

    optimizer.step()
    if n % 10 == 0:
        scheduler.step()
    losses.append(loss.item())
    n += 1
  return np.mean(losses)

def validation(model, loss_fn):
  model.eval()
  losses = []
  for values in tqdm_notebook(val_loader):
    x, features, target = values
    x, features, target = x.cuda(), features.cuda(), target.cuda()
    target = target.unsqueeze(-1)
    pred = model(x, features)
    loss = loss_fn(pred.float(), target.float())
    losses.append(loss.item())
  return np.mean(losses)

In [ ]:
loss_fn = torch.nn.MSELoss()
model = CNNModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=7e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1e-5)

In [ ]:
train_losses = []
 
for epoch in range(5):
  train_loss = train(model, optimizer, scheduler, loss_fn)
  train_losses.append(train_loss)
  print(f'{epoch} epoch, train locc: {train_loss}')

In [ ]:
plt.plot(train_losses)

# Test

In [ ]:
test_ds = ImagePandasDataset(test_pd, 'Id', img_dir=test_images_path, 
                             features_columns=FEATURES, transform=img_transforms)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

In [ ]:
model.eval()
preds = []
for image, features, _ in test_loader:
    preds += [100 * el.item() for el in model(image.cuda(), features.cuda())]

In [ ]:
sample_df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sample_df['Pawpularity'] = preds
sample_df.to_csv('submission.csv',index=False)